In [ ]:
# Snowpark Libs
from snowflake.snowpark.version import VERSION
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark.functions import col,to_date, udf
# Data Science Libs
import pandas as pd
import os
import dotenv
from pathlib import Path
from datetime import datetime
import numpy as np
import datetime
import pandas as pd

In [9]:
# Read config parser .ini file with your connection information
def connect_snowflake(Path: str) -> dict[str, str | None]:
    # Load the .env
    """ 
    Load the .env file with the connection parameters
    
    """
    # Path = r"/Users/chitrakumarsaichv/Library/CloudStorage/OneDrive-kochind.com/Desktop/Projects/purchasing_anomaly/.env"
    dotenv.load_dotenv(Path)

    # SSO - KochID
    connection_parms = {
        "account": os.getenv('account'),
        "user": os.getenv('email'),
        "authenticator": os.getenv('authenticator'),
        "database": os.getenv('database'),
        "schema": os.getenv('schema'),
        "warehouse": os.getenv('warehouse'),"role": os.getenv('role')
    } 

    connection_parms    

    # Create Snowflake Session object
    session = Session.builder.configs(connection_parms).create()
    session.sql_simplifier_enabled = True

    snowflake_environment = session.sql('SELECT current_user(), current_version()').collect()
    snowpark_version = VERSION
    
    return connection_parms

In [10]:
connect_snowflake("/Users/chitrakumarsaichv/Library/CloudStorage/OneDrive-kochind.com/Desktop/Projects/purchasing_anomaly/.env")

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://auth.kochid.com/idp/SSO.saml2?SAMLRequest=nZLBbuIwEIZfJfKeYzsBxK4FVBRaFYndRiXdQ29uPICFY2c9DilvvyYUqXtoD3uLnG%2FGv%2Bebyc1bbZIjeNTOTklGOUnAVk5pu5uS5%2FI%2B%2FU4SDNIqaZyFKTkBkpvZBGVtGjFvw94%2BwZ8WMCSxkUXR%2F5iS1lvhJGoUVtaAIlRiM%2F%2B5FjnlovEuuMoZ8qHk6wqJCD7EhNcShTrG24fQCMa6rqPdgDq%2FYznnnPEfLFJn5NuVf4tv%2BoTPGB%2Be%2BUhEvHjPdqvtZQRfxXq9QCgeyrJIi8dNSZL5NerCWWxr8BvwR13B89P6EgBjAm2POg41DkLRFlOQGNKcyg4pWtdtjTxA5eqmDbE7jV9sC4oZt9NxAKvllDQHrQbLUrpisXhV273Pt%2BWdOe3GSw8Pxdq%2FrMaj2zEsdnfj0fDgKpL8vhrOz4ZXiC2s7NlriEc8H6U8T%2FmozEYizwTP6XAwfiHJMnrVVoa%2B8hpeRuk0Nt1r1WfTqmGbzSM9a8zJZTVEf4Of%2Fe%2BDJ%2Bxjl%2Fdt%2BxUFrJaFM7o6JffO1zJ87iejWX%2BiVbrtUQG11GaulAfE6MkY1y08yBCXOvgWCJtdbv13rWd%2FAQ%3D%3D&RelayState=ver%3A1-hint%3A759822674653786-ETMsDgAAAZ

You might have more than one threads sharing the Session object trying to update sql_simplifier_enabled. Updating this while other tasks are running can potentially cause unexpected behavior. Please update the session configuration before starting the threads.


{'account': 'invistaprod.us-east-2.aws',
 'user': 'chitrakumarsai.chenurivenkata@invista.com',
 'authenticator': 'externalbrowser',
 'database': 'DS_D_WORKSPACE',
 'schema': 'THIRD_PARTY_PATENT',
 'warehouse': 'DS_D_TRAINING_WH',
 'role': 'DS_D_SCIENTIST_FR'}

In [7]:
# session.sql("CREATE SCHEMA IF NOT EXISTS PURCHASING_ANOMALY;").collect()